In [1]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"


In [30]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "azerty92")

with GraphDatabase.driver(URI, auth=AUTH) as driver: 
    driver.verify_connectivity() 

In [3]:
def create_person(tx, name):  
    result = tx.run(
        "MERGE (:Person {name: $name, school:'Telecom Paris'})",  
        name=name  
    )
    summary = result.consume()
    return summary

In [31]:
with driver.session(database="foo") as session:
    summary = session.execute_write(create_person, name="Guillaume")  

    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))

Created 0 nodes in 3 ms.


In [39]:
def count_row(tx):  
    result = tx.run(
        """
        LOAD CSV FROM 'file:///20230101000000.translation.export.csv' AS row FIELDTERMINATOR '\t'
        RETURN COUNT(row)
        """ 
    )
    count = result.fetch(1)
    summary = result.consume()
    return count, summary

def create_node_export(tx):  
    result = tx.run(
        """
        LOAD CSV FROM 'file:///20230101000000.translation.export.csv' AS row FIELDTERMINATOR '\t'
        MERGE (event:Event {GlobalEventID:row[0]})
        ON CREATE 
        SET 
            event.Date = row[1],
            event.Type = row[26]
        ;
        """ 
    )
    summary = result.consume()
    return summary

In [35]:
with driver.session(database="foo") as session:
    count, summary = session.execute_write(count_row)  

    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))
    print(count)

Created 0 nodes in 14 ms.
[<Record COUNT(row)=332>]


In [40]:
with driver.session(database="foo") as session:
    summary = session.execute_write(create_node_export)  

    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))

Created 0 nodes in 211 ms.
